In [2]:
import pandas as pd
from helper import get_my_dataset

main_df = get_my_dataset()

In [3]:
main_df.shape

(2070, 32)

In [4]:
main_df.dtypes

Accident_Index                                  object
Location_Easting_OSGR                            int64
Location_Northing_OSGR                           int64
Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Date                                            object
Day_of_Week                                      int64
Time                                            object
Local_Authority_District                         int64
Local_Authority_Highway                         object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                        int64
Speed_limit                                      int64
Junction_D

In [ ]:
numerical_cols = [
       "Number_of_Vehicles",
       "Number_of_Casualtiess",
       "Longitude",
       "Latitude",
]

categorical_cols = [
    "Speed_limit",
    "Road_Type",
    "Light_Conditions",
    "Weather_Conditions",
    "Day_of_Week",
    "Urban_or_Rural_Area"
]

def correct_data_types(df: pd.DataFrame, num_cols: list, cat_cols: list) -> pd.DataFrame:
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')
    return df


